# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [18]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2024-04-22	Metal-ring interactions in group 2 ansa-metallocenes: assessed with the local vibrational mode theory	Phys. Chem. Chem. Phys.		"Antonio, Juliana J. and Kraka, Elfi. ""Metal-ring interactions in group 2 ansa-metallocenes: assessed with the local vibrational mode theory"" <i>Phys. Chem. Chem. Phys.</i>, <b>2024</b>, <i>26</i>, 15143-15155"	Antonio2024	http://academicpages.github.io/files/paper1.pdf	http://academicpages.github.io/files/slides1.pdf
2023-12-02	Iron-histidine bonding in bishistidyl hemoproteins - A local vibrational mode study	J. Comput. Chem.		"Friendorf, Marek; Antonio, Juliana J.; Kraka, Elfi. ""Iron-histidine bonding in bishistidyl hemoproteins - A local vibrational mode study"" <i> J. Comput. Chem.</i>, <b>2024</b>, <i>45</i>, 574-588"	Freindorf2024	http://academicpages.github.io/files/paper2.pdf	http://academicpages.github.io/files/slides1.pdf
2023-05-15	Reaction mechanism - explored with th

## Import pandas

We are using the very handy pandas library for dataframes.

In [19]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [26]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-04-22,Metal-ring interactions in group 2 ansa-metall...,Phys. Chem. Chem. Phys.,NaN,"Antonio, Juliana J. and Kraka, Elfi. ""Metal-ri...",Antonio2024,http://academicpages.github.io/files/paper1.pdf,http://academicpages.github.io/files/slides1.pdf
1,2023-12-02,Iron-histidine bonding in bishistidyl hemoprot...,J. Comput. Chem.,NaN,"Friendorf, Marek; Antonio, Juliana J.; Kraka, ...",Freindorf2024,http://academicpages.github.io/files/paper2.pdf,http://academicpages.github.io/files/slides1.pdf
2,2023-05-15,Reaction mechanism - explored with the unified...,Chem. Commun.,NaN,"Kraka, Elfi; Antonio, Juliana J.; Freindorf, M...",Kraka2023,http://academicpages.github.io/files/paper3.pdf,http://academicpages.github.io/files/slides1.pdf
3,2023-09-29,Hydrogen Sulfide Ligation in Hemoglobin I of L...,J. Phys. Chem. A,NaN,"Friendorf, Marek; Antonio, Juliana J.; Kraka, ...",Freindorf2023,http://academicpages.github.io/files/paper4.pdf,http://academicpages.github.io/files/slides1.pdf
4,2023-07-21,<i>Ortho</i>-Fluoro Effect on the C-C Bond Act...,Organometallics,NaN,"Lachaize, Sebastien; Gallegos, Dominique C.; A...",Lachaize2023,http://academicpages.github.io/files/paper5.pdf,http://academicpages.github.io/files/slides1.pdf
5,2023-07-17,Noncovalent pi Interaction in Mutated Aquomet-...,Biochemistry,NaN,"Antonio, Juliana J. and Kraka, Elfi ""Noncovale...",Antonio2023,http://academicpages.github.io/files/paper6.pdf,http://academicpages.github.io/files/slides1.pdf
6,2022-12-06,Automatic Generation of Local Vibrational Mode...,J. Phys. Chem. A,NaN,"Moura, Renaldo T.; Quintano, Mateus; Antonio, ...",Moura2022,http://academicpages.github.io/files/paper7.pdf,http://academicpages.github.io/files/slides1.pdf
7,2022-11-08,The Local Vibrational Mode Theory and Its Plac...,J. Phys. Chem. A,NaN,"Kraka, Elfi; Quintano, Mateus; Antonio, Julian...",Kraka2022,http://academicpages.github.io/files/paper8.pdf,http://academicpages.github.io/files/slides1.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [28]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [29]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [30]:
!ls ../_publications/

2022-11-08-Kraka2022.md     2023-07-21-Lachaize2023.md
2022-12-06-Moura2022.md     2023-09-29-Freindorf2023.md
2023-05-15-Kraka2023.md     2023-12-02-Freindorf2024.md
2023-07-17-Antonio2023.md   2024-04-22-Antonio2024.md


In [25]:
!cat ../_publications/2024-04-22-Antonio2024.md

---
title: "Metal-ring interactions in group 2 ansa-metallocenes: assessed with the local vibrational mode theory"
collection: publications
permalink: /publication/2024-04-22-Antonio2024
date: 2024-04-22
venue: 'Phys. Chem. Chem. Phys.'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Antonio, Juliana J. and Kraka, Elfi. &quot;Metal-ring interactions in group 2 ansa-metallocenes: assessed with the local vibrational mode theory&quot; <i>Phys. Chem. Chem. Phys.</i>, <b>2024</b>, <i>26</i>, 15143-15155'
---
[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Antonio, Juliana J. and Kraka, Elfi. "Metal-ring interactions in group 2 ansa-metallocenes: assessed with the local vibrational mode theory" <i>Phys. Chem. Chem. Phys.</i>, <b>2024</b>, <i>26</i>, 15143-15155